# Ring Detection

To detect the rings generated in the previous steps, we use a clustering algorithm, which will give us an estimate of their centers and radii

The data we want to extract from the csv files are the PointsSet class instaces, to later calculate error, and just a list of all points, to make the fuzzy c-means algorithm. We will use this algorithm because it allows us to assign a membership degree for each cluster to a point, instead of only assigning it to one.

In [103]:
DATASET_LOCATION = "../dataset" # Specifies where the dataset is located
OUTPUT_DIRECTORY = "../results" # Specifies where the results of the ring detection will be saved
FUZZINESS_RANGE = (1.05, 2.0)
ATTEMPTS = 40 # Number of times we will run the algorithm per csv file
MAX_ITERATIONS = 100 # Maximum number of iterations to find the center points
MEMBERSHIP_THRESSHOLD = 0.3 # Minimum membership degree for a point to be considered belonging to a cluster (0.0 - 1.0), used to calculate radius

In [104]:
import math, random, os, datetime, json
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
from tqdm.notebook import tqdm

## PointsSet Class

We will use this class again here to help parse the csv and reconstruct the data 

In [105]:
class PointsSet:
    def __init__(self, points, center, radius, circ_no):
        self.points = points
        self.center = center if not math.isnan(circ_no) else None
        self.radius = radius if not math.isnan(circ_no) else None
        self.circ_no = int(circ_no) if not math.isnan(circ_no) else None

    def add_point(self, point):
        self.points.append(point)

    def is_noise(self):
        return self.circ_no is None

    def unpack(self):
        if self.is_noise():
            return [[p[0], p[1], None, None, None] for p in self.points]
        else:
            return [[p[0], p[1], self.center[0], self.center[1], self.radius, self.circ_no] for p in self.points]

    def __str__(self):
        if self.is_noise():
            return f"{len(self.points)} of Noise"
        else:
            return f"Circunference {self.circ_no} has {len(self.points)} points and center in {self.center}"

## Data extraction

The first step to represent the data is to extract it from the csv files. This time we have to take care of NaN values since comparing it to others will result on each noise point to be on its own set of points.

In [106]:
def extract_point_sets(df):
    data = []
    for index, row in df.iterrows():
        
        existing_ps = next(filter(lambda ps: ps.circ_no==row.circ_no if not math.isnan(row.circ_no) else ps.circ_no is None, data), None)
        if existing_ps is not None:
            existing_ps.add_point((row.point_x, row.point_y))
        else:
            data.append(PointsSet([(row.point_x, row.point_y)], (row.center_x, row.center_y), row.radius, row.circ_no))

    points = []
    for points_set in data:
        points.extend(points_set.points)
    
    rings = sorted(list(filter(lambda x : not x.is_noise() , data)), key=lambda x: x.circ_no)
    noise = [ps.points for ps in list(filter(lambda x : x.is_noise() , data))]
    return (points, rings, noise)

# Fuzzy C-Means Algorithm

We define here the fuzzy c-means algorithm, and starting with a random ammount of centers, we will make a certain ammount of iterations until the centers do not change.

For this specific problem, if a center is found to have no points assigned to its cluster, we relocate it randomly, because we know we have exactly k rings, and we wish to find k centers

Apart from the parameters we take in the k-means algorithm, we also take "m" as a parameter. This value defines how much sharing of data points we want to allow.

We will run this algorithm until the center points do not change or when we reach se specified ammount of maximum iterations

In [107]:
def alg_iteration(centers, points, m):
    # m is the fuzziness parameter, usually set to 2
    fuzziness = 2 / (m - 1)
    n_clusters = len(centers)
    n_points = len(points)
    membership_matrix = np.zeros((n_points, n_clusters))
    
    # compute the membership degree of each point to each cluster
    for i in range(n_points):
        for j in range(n_clusters):
            distance = np.sqrt((centers[j][0]-points[i][0])**2 + (centers[j][1]-points[i][1])**2)
            membership_matrix[i][j] = 1 / ((sum([(distance/np.sqrt((centers[k][0]-points[i][0])**2 + (centers[k][1]-points[i][1])**2))**(fuzziness) for k in range(n_clusters)])))
    
    # update the cluster centroids based on the membership degrees
    new_centers = [(0,0) for _ in range(n_clusters)]
    for j in range(n_clusters):
        new_centers[j] = tuple(np.average(points, axis=0, weights=membership_matrix[:, j]))
    return new_centers, membership_matrix

In [108]:
def alg_perform(k, points):
    old_centers = [(random.uniform(0.0, 100.0), random.uniform(0.0, 100.0)) for _ in range(k)]
    new_centers, membership_matrix = alg_iteration(old_centers, points, random.uniform(*FUZZINESS_RANGE))
    count = 1
    while(count<=MAX_ITERATIONS and old_centers!=new_centers):
        old_centers = new_centers
        new_centers, membership_matrix = alg_iteration(old_centers, points, random.uniform(*FUZZINESS_RANGE))
        count +=1
    return new_centers, membership_matrix

In [109]:
''' K-Means version
def alg_iteration(centers, points):
    new_centers = [(0,0) for _ in range(len(centers))]
    clusters_points_mapping = {}
    [clusters_points_mapping.setdefault(i, []) for i in range(len(centers))]
    for p in points:
        best_distance_to_center = None
        for i in range(len(centers)):
            distance_to_center = math.sqrt((centers[i][0]-p[0])**2 + (centers[i][1]-p[1])**2)
            nearest_center = i if best_distance_to_center is None or distance_to_center < best_distance_to_center else nearest_center
            best_distance_to_center = distance_to_center if best_distance_to_center is None else min(best_distance_to_center, distance_to_center)
        clusters_points_mapping[nearest_center].append(p)
    
    for index in clusters_points_mapping:
        if(len(clusters_points_mapping[index])==0):
            new_centers[index] = (random.uniform(0.0, 100.0), random.uniform(0.0, 100.0))
        else:
            new_centers[index] = (np.median([p[0] for p in clusters_points_mapping[index]]), np.median([p[1] for p in clusters_points_mapping[index]]))

    return new_centers
'''

' K-Means version\ndef alg_iteration(centers, points):\n    new_centers = [(0,0) for _ in range(len(centers))]\n    clusters_points_mapping = {}\n    [clusters_points_mapping.setdefault(i, []) for i in range(len(centers))]\n    for p in points:\n        best_distance_to_center = None\n        for i in range(len(centers)):\n            distance_to_center = math.sqrt((centers[i][0]-p[0])**2 + (centers[i][1]-p[1])**2)\n            nearest_center = i if best_distance_to_center is None or distance_to_center < best_distance_to_center else nearest_center\n            best_distance_to_center = distance_to_center if best_distance_to_center is None else min(best_distance_to_center, distance_to_center)\n        clusters_points_mapping[nearest_center].append(p)\n    \n    for index in clusters_points_mapping:\n        if(len(clusters_points_mapping[index])==0):\n            new_centers[index] = (random.uniform(0.0, 100.0), random.uniform(0.0, 100.0))\n        else:\n            new_centers[index] 

# Radii Estimation

Now that we have calculated the centers, we need to estimate the radii of the circunferences. For that we will take into account the membership degree and thresshold defined to consider a point top be part of a cluster.

In [110]:
def estimate_radii(center, points, center_membership_matrix):
    member_weights = center_membership_matrix[center_membership_matrix >= MEMBERSHIP_THRESSHOLD]
    member_points = np.asarray(points)[center_membership_matrix >= MEMBERSHIP_THRESSHOLD]
    return np.average([np.sqrt((center[0]-p[0])**2 + (center[1]-p[1])**2) for p in member_points], weights=member_weights)
    

# Error

The plan is to execute the algorithm more than once per sample, and then choose the best estimation out of all of them. For that, we need to calculate the error, meaning the "distance" between the current estimated centers and the actual centers themselves.

We will also take into account the radii of the rings, however these will be considered on a different level, because we can have good centers and bad radii (circle inside of another), or bad centers but good radii.

So that is why we will give 2 error values, one for the distance between the expected center and estimated one, and one for the difference between the expected and predicted radii.
 
In order to calculate both error values, we need to first relate each of the predicted centers with each real center, for that we have to select 3 pairs such as the total distance is minimal, and then we can use that same distance for the error calculation. To find these 3 pairs we will use a matrix which will contain the cost of a certain pair (distance in this case), then we can solve it as a linear optimization problem finding the minumim combination of values in the matrix choosing only 1 value per row and column. We can do this last part with the linear_sum_assignment() function of scipy.

The error we will return for both estamtions will be an average of the errors for each circunference, in a value from 0.0 to 1.0:
- In the case of centers: 1.0 being sqrt(100² + 100²) (sqrt(2000)) the distance between the estimate and the real center and 0.0 being exact
- In the case of radii: 1.0 being the real radius value the difference between the size of the real radius and estimated one, and 0.0 being exact

We have, however, to give one single value of error for us to make the comparison between different estimations for the same cloud of points, we will then give more value to the centers than to the radii.

In [111]:
def find_pairs(centers, rings_centers):
    cost_matrix = np.zeros((len(centers), len(rings_centers)))
    
    for i in range(len(centers)):
        for j in range(len(rings_centers)):
            cost_matrix[i][j] = np.sqrt((centers[i][0] - rings_centers[j][0])**2 + (centers[i][1] - rings_centers[j][1])**2)
    center_ind, ring_ind = linear_sum_assignment(cost_matrix)

    return [[int(center_ind[i]), int(ring_ind[i])] for i in range(len(center_ind))]

In [112]:
def get_centers_error(pairs, centers, rings):
    avg_offset = np.mean([np.sqrt((centers[p[0]][0] - rings[p[1]].center[0])**2 + (centers[p[0]][1] - rings[p[1]].center[1])**2) for p in pairs])
    return avg_offset / np.sqrt(2000)

In [113]:
def get_radii_error(pairs, centers, rings, points, membership_matrix):
    return np.mean([abs(estimate_radii(centers[p[0]], points, membership_matrix[:, p[0]]) - rings[p[1]].radius)/rings[p[1]].radius for p in pairs])

In [114]:
def get_error(centers_error, radii_error):
    return centers_error * 0.9 + radii_error * 0.1

In [115]:
def get_total_error(centers, rings, points, membership_matrix):
    pairs = find_pairs(centers, [r.center for r in rings])
    return get_centers_error(pairs, centers, rings)*0.9 + get_radii_error(pairs, centers, rings, points, membership_matrix)*0.1

# Main Loop

With all the previous functions, now its time to run the algorithm. But before that we will define and create the data structure to save all the dataset information and predicted data, so that we only have to extract information from one json file when studying the results. 

Initially, we do not do this with the dataset because we prefer it to be more readable and flexible, which a folder based structure provides. However when it comes to reading results, if we do not have everything we need in one place, we might risk losing information later down the road, for example, if the dataset changes.

In [ ]:
results = {}
'''
{
    set_type: 
    {
        filename:
        {
            circs_num = int,
            circunferences:
            {
                cir_no:
                {
                    points: [],
                    center: [],
                    radius: []
                }
            }
            noise: [],
            pairs: [],
            predicted_centers: [],
            predicted_radii: [],
            centers_error: float,
            radii_error: float,
            tot_error: float
        }
    }  
}
'''

for set_type in ["clean", "extends", "collides"]:
    results[set_type] = {}
    for filename in tqdm(os.listdir(f"{DATASET_LOCATION}/{set_type}"), desc=f"Predicting clouds of type {set_type}", leave=True):
            if filename.endswith(".csv"): 
                df = pd.read_csv(f"{DATASET_LOCATION}/{set_type}/{filename}",header=0, sep=";")
                points, rings, noise = extract_point_sets(df)
                predicted_centers, membership_matrix = min([alg_perform(len(rings), points) for _ in tqdm(range(ATTEMPTS), desc=f"Predicting {filename}", leave=True)], key=lambda c: get_total_error(c[0], rings, points, c[1]))

                pairs = find_pairs(predicted_centers, [r.center for r in rings])
                centers_error = get_centers_error(pairs, predicted_centers, rings)
                radii_error = get_radii_error(pairs, predicted_centers, rings, points, membership_matrix)
                results[set_type][filename] =   {
                                                "circs_num": len(rings),
                                                "circunferences":
                                                {
                                                    f"{r.circ_no}":
                                                    {
                                                        "points": r.points,
                                                        "center": r.center,
                                                        "radius": r.radius
                                                    }
                                                    for r in rings
                                                },
                                                "noise": noise,
                                                "pairs": pairs, 
                                                "predicted_centers": predicted_centers, 
                                                "predicted_radii": [estimate_radii(predicted_centers[p[0]], points, membership_matrix[:, p[0]]) for p in pairs],
                                                "centers_error": centers_error,
                                                "radii_error": radii_error,
                                                "tot_error": get_error(centers_error, radii_error)
                                                }

# Save Results

We finally save the results to a json, in a folder specified by the user.

In [117]:
current_moment = datetime.datetime.now()
if not os.path.exists(f"{OUTPUT_DIRECTORY}/{current_moment}"):
            os.makedirs(f"{OUTPUT_DIRECTORY}/{current_moment}")
with open(f"{OUTPUT_DIRECTORY}/{current_moment}/results.json", 'w') as outfile:
    json.dump(results, outfile)